In [1]:

import os
import json
import pandas as pd
import geopandas as gpd
import re

### Read the downloaded/nonspatial and spatial data

In [2]:

os.chdir('..')
home_dir = os.getcwd()
downloaded_data = r"downloaded-data"


In [3]:
# replace... blds_continents_data_18-23_ai.json blds_africa_states_18-23_ai.json
with open(f"{downloaded_data}\\blds_africa_states_18-23_ai.json", "r") as f:
    nonspatial_d = json.load(f)
# nonspatial_d



In [4]:
os.getcwd()

'c:\\Users\\milan\\OneDrive - MUNI\\VŠ\\PhD\\Zahraniční stáž\\Work\\HeiGIT_notebooks\\analysis\\ai-assisted-osm-mapping-stats'

In [5]:
shp = r"shp"
s_df = gpd.read_file(f"{shp}/africa-states-borders.shp")

#### Edit the obtained data into desired shape to create a DataFrame

In [6]:
d = {
    "feature": [],
    "source": [],
    "timestamp": [],
    "value": []
}


In [7]:
for cont in nonspatial_d:
    for dic1 in cont:
        feature, source = dic1["groupByObject"][0], dic1["groupByObject"][1]
        for res in dic1["result"]:
            d["feature"].append(feature)
            d["source"].append(source)
            d["timestamp"].append(res["timestamp"])
            d["value"].append(res["value"])

ns_df = pd.DataFrame(d)
# ns_df

In [8]:
# xtract only the year
ns_df["year"] = pd.to_datetime(ns_df["timestamp"]).dt.year

In [9]:
ns_df

,feature,source,timestamp,value,year
0,0,remainder,2018-01-01T00:00:00Z,63257.0,2018
1,0,remainder,2019-01-01T00:00:00Z,82349.0,2019
2,0,remainder,2020-01-01T00:00:00Z,177419.0,2020
3,0,remainder,2021-01-01T00:00:00Z,371148.0,2021
4,0,remainder,2022-01-01T00:00:00Z,544453.0,2022
...,...,...,...,...,...
607,9,source=microsoft/BuildingFootprints,2019-01-01T00:00:00Z,0.0,2019
608,9,source=microsoft/BuildingFootprints,2020-01-01T00:00:00Z,0.0,2020
609,9,source=microsoft/BuildingFootprints,2021-01-01T00:00:00Z,0.0,2021
610,9,source=microsoft/BuildingFootprints,2022-01-01T00:00:00Z,0.0,2022


#### Append the names of the regions to the DataFrame

In [10]:

# replace... geojson-continent-borders geojson-africa-states
geojson_dir = os.path.join(home_dir, "geojson-africa-states", "")

geojson_names = []
for (dir_path, dir_names, file_names) in os.walk(geojson_dir):  # give the path
    geojson_names.extend(file_names)
print(geojson_names)
print(os.getcwd())


['0_Burundi.geojson', '10_Djibouti.geojson', '11_Algeria.geojson', '12_Egypt.geojson', '13_Gabon.geojson', '14_Eritrea.geojson', '15_Ghana.geojson', '16_Ethiopia.geojson', '17_Guinea.geojson', '18_Gambia.geojson', '19_Guinea Bissau.geojson', '1_Benin.geojson', '20_Morocco.geojson', '21_Liberia.geojson', '22_Lesotho.geojson', '23_Equatorial Guinea.geojson', '24_Libya.geojson', '25_Madagascar.geojson', '26_Kenya.geojson', '27_Mozambique.geojson', '28_Mauritania.geojson', '29_Niger.geojson', '2_Burkina Faso.geojson', '30_Rwanda.geojson', '31_Western Sahara.geojson', '32_Nigeria.geojson', '33_Sudan.geojson', '34_Malawi.geojson', '35_Namibia.geojson', '36_Senegal.geojson', '37_South Sudan.geojson', '38_Mali.geojson', '39_Somaliland.geojson', '3_Ivory Coast.geojson', '40_Swaziland.geojson', '41_Chad.geojson', '42_Somalia.geojson', '43_Sierra Leone.geojson', '44_Togo.geojson', '45_Tunisia.geojson', '46_Uganda.geojson', '47_United Republic of Tanzania.geojson', '48_South Africa.geojson', '49_Z

In [11]:
geojson_names_2 = geojson_names

id_name_dic = {int(re.search(r'^\d+', k)[0]): re.search(r'\_.+\.', v)[0][1:-1]
                for k, v in zip(geojson_names, geojson_names_2)}
id_name_ser = pd.Series(id_name_dic)
id_name_df = pd.DataFrame(id_name_ser, columns=["name"])
id_name_df = id_name_df.reset_index().rename(columns={"index": "id"})
id_name_df["id"] = id_name_df["id"].astype(int)
# id_name_df


In [12]:
# merge df with region names based on ids
ns_df = ns_df.merge(id_name_df, how="left", left_on="feature",
               right_on="id")
ns_df.head()


,feature,source,timestamp,value,year,id,name
0,0,remainder,2018-01-01T00:00:00Z,63257.0,2018,0,Burundi
1,0,remainder,2019-01-01T00:00:00Z,82349.0,2019,0,Burundi
2,0,remainder,2020-01-01T00:00:00Z,177419.0,2020,0,Burundi
3,0,remainder,2021-01-01T00:00:00Z,371148.0,2021,0,Burundi
4,0,remainder,2022-01-01T00:00:00Z,544453.0,2022,0,Burundi


In [13]:
ns_df_sel = ns_df.groupby("name")[["value", "year"]].last().reset_index()
ns_df_sel.head()

,name,value,year
0,Algeria,478.0,2023
1,Angola,513.0,2023
2,Benin,439.0,2023
3,Botswana,9.0,2023
4,Burkina Faso,1088.0,2023


In [14]:
s_df.head(2)

,scalerank,featurecla,labelrank,sovereignt,sov_a3,adm0_dif,level,type,admin,adm0_a3,...,region_un,subregion,region_wb,name_len,long_len,abbrev_len,tiny,homepart,filename,geometry
0,1,Admin-0 country,6,Burundi,BDI,0,2,Sovereign country,Burundi,BDI,...,Africa,Eastern Africa,Sub-Saharan Africa,7,7,4,-99,1,BDI.geojson,"POLYGON ((29.34000 -4.49998, 29.27638 -3.29391..."
1,1,Admin-0 country,5,Benin,BEN,0,2,Sovereign country,Benin,BEN,...,Africa,Western Africa,Sub-Saharan Africa,5,5,5,-99,1,BEN.geojson,"POLYGON ((2.69170 6.25882, 1.86524 6.14216, 1...."


In [15]:
merged_data = s_df.merge(ns_df_sel, how="left", left_on="sovereignt", right_on="name")
merged_data.head(2)

,scalerank,featurecla,labelrank,sovereignt,sov_a3,adm0_dif,level,type,admin,adm0_a3,...,name_len,long_len,abbrev_len,tiny,homepart,filename,geometry,name_y,value,year
0,1,Admin-0 country,6,Burundi,BDI,0,2,Sovereign country,Burundi,BDI,...,7,7,4,-99,1,BDI.geojson,"POLYGON ((29.34000 -4.49998, 29.27638 -3.29391...",Burundi,1517.0,2023
1,1,Admin-0 country,5,Benin,BEN,0,2,Sovereign country,Benin,BEN,...,5,5,5,-99,1,BEN.geojson,"POLYGON ((2.69170 6.25882, 1.86524 6.14216, 1....",Benin,439.0,2023
